In [1]:

## impports very important libraries run this cell first (TeaToCodeConverter)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
import sweetviz as sv
#import tensorflow as tf


/home/scv1alpha/.local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data Imports

importing unlabeled  data as U1
importing labeled data as L1

In [9]:
## creating a custom missing value list
missing_values = ["NA","N/a",np.nan,"?"]


u1 = pd.read_csv("DataSets/Breast cancer dataset/breast-cancer-wisconsin.data",header=None,na_values=missing_values)
l1 = pd.read_csv("DataSets/Breast cancer dataset/breast-cancer-wisconsin.data",header=None,na_values=missing_values, names=['id','clump_thickness','uniformity_of_cell_size','uniformity_of_cell_shape','marginal_adhesion','single_epithelial_cell_size','bare_nuclei','bland_chromatin','normal_nucleoli','mitoses','diagnosis'])

##convert the bare_nuclei colmn to a number and drop the rows
#l1['bare_nuclei'] = pd.to_numeric(l1['bare_nuclei'],errors='coerce')

#check data is clean
l1.isnull().sum()
l1.head()

print(l1.dtypes)


FileNotFoundError: [Errno 2] No such file or directory: 'DataSets/Breast cancer dataset/breast-cancer-wisconsin.data'

In [ ]:
l1.dtypes

In [ ]:
#Clean up the data
l1['diagnosis'].replace({2:0,4:1},inplace=True)
l1
#analysis = sv.analyze(l1)
#analysis.show_html('EDA-Sweetviz2predrop.html', open_browser=False)

Sweetvis summary



In [ ]:
# split the dataset
x=l1.drop('diagnosis',axis=1)
y=l1['diagnosis']
x



In [ ]:
# Summary of the dataset

#analysis = sv.analyze(l1, target_feat='diagnosis')
#analysis.show_html('EDA-Sweetviz2.html', open_browser=False)


In [ ]:
#Dislay resuls in the notebook
import IPython
ip = IPython.display.IFrame(src='EDA-Sweetviz2.html', width=1000, height=600)


# using Sklearn to diagnosisify the diagnosis data and plot some graphs

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder_y = LabelEncoder()
l1.diagnosis = labelencoder_y.fit_transform(l1.diagnosis.values)

sns.pairplot(l1, hue='diagnosis')

#reencode entire dataframe
l1 = l1.apply(LabelEncoder().fit_transform)

In [ ]:
#get the correlation of the columns
l1.corr()

#visualize the correlation
plt.figure(figsize=(10,10))
sns.heatmap(l1.corr(), annot=True, fmt='.0%')

# Splitting the data into training and testing sets

In [ ]:
# split the data into independent 'X' and dependent 'Y' variables

X=l1.drop('diagnosis',axis=1)

Y=l1['diagnosis']

# Splitting the data into training and testing sets


from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)


In [ ]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [ ]:
#without scaling funtion for the models
def models(X_train,Y_train):
    
    #Logistic Regression
    from sklearn.linear_model import LogisticRegression
    log = LogisticRegression(random_state = 0)
    log.fit(X_train, Y_train)
    
    #Decision Tree
    from sklearn import tree
    from sklearn.tree import DecisionTreeClassifier
    dtc = DecisionTreeClassifier(criterion = 'entropy',random_state=0)
    dtcRender = dtc.fit(X_train, Y_train) 

    
    #Random Forest diagnosisifier
    from sklearn.ensemble import RandomForestClassifier
    forest = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
    forest.fit(X_train, Y_train)

    #print model accuracy on the training data.

    print('[0]Logistic Regression Training Accuracy:', log.score(X_train, Y_train))
    print('[1]Decision Tree diagnosisifier Training Accuracy:', dtc.score(X_train, Y_train))
    print('[2]Random Forest diagnosisifier Training Accuracy:', forest.score(X_train, Y_train))

    return log, dtc, dtcRender, forest




In [ ]:
#Getting all of the models
model = models(X_train, Y_train)

In [ ]:
#test model accuracy on test data on confusion matrix for the models

from sklearn.metrics import confusion_matrix
for i in model:
    print('Model ', i)
    cm = confusion_matrix(Y_test, i.predict(X_test))
    #TN, FP, FN, TP variables
    TP = cm[0][0]
    TN = cm[1][1]
    FP = cm[0][1]
    FN = cm[1][0]

    print(cm)
    ## really important to formula it is the accuracy of the model
    print('Testing Accuracy = "{}!"'.format( (TP + TN) / (TP + TN + FN + FP)))
    

In [ ]:
#testing from other libaries

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

for i in range(len(model)):
    print('Model ', i)
    # Check precision, recall, f1-score
    print( classification_report(Y_test, model[i].predict(X_test)) )
    # Another way to get the models accuracy on the test data
    print( accuracy_score(Y_test, model[i].predict(X_test)))
    print()

#classisify the data using the random forest classisifier

In [ ]:
pred=model[2].predict(X_test)
print(pred.tolist())
print()
print(Y_test.tolist())


In [ ]:
#visualizing the desicion tree
from IPython.display import Image
from six import StringIO
from sklearn.tree import export_graphviz
import pydotplus

features = list(l1.columns[1:])

## export to a file (png,pdf)
#export_graphviz(model[1],out_file='dtree.png')

In [ ]:
#print(dtc)
print(dir(model[1]))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import graphviz

#tree.plot_tree(model[1])
#tree.png
## plot the tree
#tree.plot_tree(model[1],feature_names=features,filled=True)



## export to a file (dot)
tree.export_graphviz(model[1],out_file='dtree.dot',feature_names=features,filled=True)
## convert the tree to a pdf

dot_data = tree.export_graphviz(model[1],out_file=None,feature_names=features,filled=True)
graph = graphviz.Source(dot_data)
#import os

graph.render("dtree_render",view=False)


# Comparison report

In [ ]:
compare_report = sv.compare([X_train, 'Train'], [X_test, 'Test'], 'diagnosis')
compare_report.show_html('EDA-Sweetviz-Compare2.html', open_browser=False)
